In [81]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

In [82]:
run_200_300 = pd.read_csv('data/packet loss changes/20211026T184846_200-300-iperf.csv')
run_200_1000 = pd.read_csv('data/packet loss changes/20211023T012231_200-750-iperf.csv')
run_200_25000 = pd.read_csv('data/packet loss changes/20211023T012231_200-750-iperf.csv')
run_200_5000 = pd.read_csv('data/packet loss changes/20211026T184847_200-5000-iperf.csv')
run_200_500 = pd.read_csv('data/packet loss changes/20211023T012228_200-500-iperf.csv')
run_200_750 = pd.read_csv('data/packet loss changes/20211023T012231_200-750-iperf.csv')
run_200_200 = pd.read_csv('data/packet loss changes/20211104T232610_200-200-iperf.csv')
run_200_400 = pd.read_csv('data/packet loss changes/20211104T232611_200-400-iperf.csv')
run_200_600 = pd.read_csv('data/packet loss changes/20211104T232612_200-600-iperf.csv')
run_200_800 = pd.read_csv('data/packet loss changes/20211105T000042_200-800-iperf.csv')
run_200_1000 = pd.read_csv('data/packet loss changes/20211105T000043_200-1000-iperf.csv')
run_200_1200 = pd.read_csv('data/packet loss changes/20211105T000045_200-1200-iperf.csv')
run_200_800 = pd.read_csv('data/packet loss changes/20211105T001218_200-800-iperf.csv')
run_200_2000 = pd.read_csv('data/packet loss changes/20211105T003333_200-2000-iperf.csv')
run_200_1400 = pd.read_csv('data/packet loss changes/20211105T003336_200-1400-iperf.csv')
run_200_1800 = pd.read_csv('data/packet loss changes/20211105T003338_200-1800-iperf.csv')
run_200_2500 = pd.read_csv('data/packet loss changes/20211105T004747_200-2000-iperf.csv')

In [83]:
def pckt_count(a):
    a = a.split(';')
    a = [int(i) for i in a[:-1]]
    return len(a)

In [84]:
run_200_300['pck_ct'] = run_200_300['packet_times'].apply(pckt_count)
run_200_1000['pck_ct'] = run_200_1000['packet_times'].apply(pckt_count)
run_200_25000['pck_ct'] = run_200_25000['packet_times'].apply(pckt_count)
run_200_5000['pck_ct'] = run_200_5000['packet_times'].apply(pckt_count)
run_200_500['pck_ct'] = run_200_500['packet_times'].apply(pckt_count)
run_200_750['pck_ct'] = run_200_750['packet_times'].apply(pckt_count)
run_200_200['pck_ct'] = run_200_200['packet_times'].apply(pckt_count)
run_200_400['pck_ct'] = run_200_400['packet_times'].apply(pckt_count)
run_200_600['pck_ct'] = run_200_600['packet_times'].apply(pckt_count)
run_200_800['pck_ct'] = run_200_800['packet_times'].apply(pckt_count)
run_200_1000['pck_ct'] =  run_200_1000['packet_times'].apply(pckt_count)
run_200_1200['pck_ct'] =  run_200_1200['packet_times'].apply(pckt_count)
run_200_800['pck_ct'] = run_200_800['packet_times'].apply(pckt_count)
run_200_2000['pck_ct'] =  run_200_2000['packet_times'].apply(pckt_count)
run_200_1400['pck_ct'] =  run_200_1400['packet_times'].apply(pckt_count)
run_200_1800['pck_ct'] = run_200_1800['packet_times'].apply(pckt_count)
run_200_2500['pck_ct'] =  run_200_2500['packet_times'].apply(pckt_count)

In [85]:
run_200_5000['packet_size_total'] = run_200_5000['1->2Pkts'] + run_200_5000['2->1Pkts']
run_200_300['packet_size_total'] = run_200_300['1->2Pkts'] + run_200_300['2->1Pkts']
run_200_1000['packet_size_total'] = run_200_1000['1->2Pkts'] + run_200_1000['2->1Pkts']
run_200_25000['packet_size_total'] = run_200_25000['1->2Pkts'] + run_200_25000['2->1Pkts']
run_200_500['packet_size_total'] =run_200_500['1->2Pkts'] + run_200_25000['2->1Pkts']
run_200_750['packet_size_total'] =run_200_750['1->2Pkts'] + run_200_25000['2->1Pkts']
run_200_200['packet_size_total'] =run_200_200['1->2Pkts'] + run_200_25000['2->1Pkts']
run_200_400['packet_size_total'] =run_200_400['1->2Pkts'] + run_200_25000['2->1Pkts']
run_200_600['packet_size_total'] =run_200_600['1->2Pkts'] + run_200_25000['2->1Pkts']
run_200_800['packet_size_total'] =run_200_800['1->2Pkts'] + run_200_25000['2->1Pkts']
run_200_1000['packet_size_total'] = run_200_1000['1->2Pkts'] + run_200_25000['2->1Pkts']
run_200_1200['packet_size_total'] = run_200_1200['1->2Pkts'] + run_200_25000['2->1Pkts']
run_200_800['packet_size_total'] = run_200_800['1->2Pkts'] + run_200_25000['2->1Pkts']
run_200_2000['packet_size_total'] = run_200_2000['1->2Pkts'] + run_200_25000['2->1Pkts']
run_200_1400['packet_size_total'] = run_200_1400['1->2Pkts'] + run_200_25000['2->1Pkts']
run_200_1800['packet_size_total'] = run_200_1800['1->2Pkts'] + run_200_25000['2->1Pkts']
run_200_2500['packet_size_total'] = run_200_2500['1->2Pkts'] + run_200_25000['2->1Pkts']

In [86]:
ratio_200_5000  = [0.04 for i in range(310)] 
ratio_200_300 = [0.67 for i in range(324)]
ratio_200_1000 = [0.2 for i in range(318)]
ratio_200_25000 = [0.008 for i in range(len(run_200_25000))]
ratio_200_500 = [0.4 for i in range(len(run_200_500))] 
ratio_200_750= [0.266 for i in range(len(run_200_750))]
ratio_200_200= [1 for i in range(len(run_200_200))]
ratio_200_400= [0.5 for i in range(len(run_200_400))]
ratio_200_600= [0.333 for i in range(len(run_200_600))]
ratio_200_800= [0.25 for i in range(len(run_200_800))]
ratio_200_1000= [0.2 for i in range(len(run_200_1000))]
ratio_200_1200= [0.166 for i in range(len(run_200_1200))]
ratio_200_800= [0.25 for i in range(len(run_200_800))]
ratio_200_2000= [0.1 for i in range(len(run_200_2000))]
ratio_200_1400= [0.142 for i in range(len(run_200_1400))]
ratio_200_1800= [0.111 for i in range(len(run_200_1800))]
ratio_200_2500= [0.08 for i in range(len(run_200_2500))]

In [87]:
run_200_5000['ratio'] = ratio_200_5000
run_200_300['ratio'] = ratio_200_300
run_200_1000['ratio'] = ratio_200_1000
run_200_25000['ratio'] = ratio_200_25000
run_200_500['ratio'] = ratio_200_500
run_200_750['ratio'] = ratio_200_750
run_200_200['ratio'] = ratio_200_200
run_200_400['ratio'] = ratio_200_400
run_200_600['ratio'] = ratio_200_600
run_200_800['ratio'] = ratio_200_800
run_200_1000['ratio'] = ratio_200_1000
run_200_1200['ratio'] = ratio_200_1200
run_200_800['ratio'] = ratio_200_800
run_200_2000['ratio'] = ratio_200_2000
run_200_1400['ratio'] = ratio_200_1400
run_200_1800['ratio'] = ratio_200_1800
run_200_2500['ratio'] = ratio_200_2500

In [88]:
#optional - take out first 25 seconds of data
run_200_5000 = run_200_5000[25:]
run_200_300 = run_200_300[25:]
run_200_1000 = run_200_1000[25:]
run_200_25000 = run_200_25000[25:]
run_200_500 = run_200_500[25:]
run_200_750 = run_200_750[25:]
run_200_200 = run_200_200[25:]
run_200_400 = run_200_400[25:]
run_200_600 = run_200_600[25:]
run_200_800 = run_200_800[25:]
run_200_2000 = run_200_2000[25:]
run_200_1400 = run_200_1400[25:]
run_200_1800 = run_200_1800[25:]
run_200_2500 = run_200_2500[25:]

In [89]:
def agg_10sec(df):
    new_df = pd.DataFrame()
    min_time = df["Time"][25]
    ratio = df["ratio"][25]
    while min_time < df["Time"][len(df)-1]:
        temp_df = df[(df["Time"] >= min_time) & (df["Time"] < min_time+10)]
        row = temp_df[["1->2Bytes", "2->1Bytes", "1->2Pkts", "2->1Pkts", "packet_size_total", "pck_ct"]].sum().to_frame().T
        row["packet_sizes_var"] = temp_df["packet_size_total"].var()
        row["Time"] = min_time
        row["ratio"] = ratio
        new_df = new_df.append(row)
        min_time += 10
    return new_df.reset_index(drop=True)

In [90]:
agg_200_300 = agg_10sec(run_200_300)
agg_200_1000 = agg_10sec(run_200_1000)
agg_200_5000 = agg_10sec(run_200_5000)
agg_200_25000 = agg_10sec(run_200_25000)
agg_200_500 = agg_10sec(run_200_500)
agg_200_750 = agg_10sec(run_200_750)
agg_200_200 = agg_10sec(run_200_200)
agg_200_400 = agg_10sec(run_200_400)
agg_200_600 = agg_10sec(run_200_600)
agg_200_800 = agg_10sec(run_200_800)
agg_200_1000 =  agg_10sec(run_200_1000)
agg_200_2000 =  agg_10sec(run_200_2000)
agg_200_1400 =  agg_10sec(run_200_1400)
agg_200_1800 =  agg_10sec(run_200_1800)
agg_200_2500 =  agg_10sec(run_200_2500)

In [91]:
full_df = pd.concat([agg_200_300, agg_200_1000, agg_200_25000])
test_df = agg_200_5000

In [92]:
full_df

,1->2Bytes,2->1Bytes,1->2Pkts,2->1Pkts,packet_size_total,pck_ct,packet_sizes_var,Time,ratio
0,1444556.0,12592.0,964.0,235.0,1199.0,1199.0,1536.600000,1635274148,0.670
1,1446000.0,15172.0,964.0,283.0,1247.0,1247.0,196.677778,1635274158,0.670
2,1441500.0,15228.0,961.0,285.0,1246.0,1246.0,173.822222,1635274168,0.670
3,1456500.0,16052.0,971.0,299.0,1270.0,1270.0,108.222222,1635274178,0.670
4,1434056.0,15088.0,957.0,280.0,1237.0,1237.0,1587.072727,1635274188,0.670
...,...,...,...,...,...,...,...,...,...
21,2753660.0,51016.0,994.0,949.0,1943.0,1943.0,3540.454545,1634952385,0.008
22,2808836.0,49744.0,997.0,937.0,1934.0,1934.0,203.822222,1634952395,0.008
23,2749520.0,51400.0,998.0,955.0,1953.0,1953.0,221.788889,1634952405,0.008
24,2814680.0,49616.0,998.0,935.0,1933.0,1933.0,282.677778,1634952415,0.008


In [93]:
###PARTIAL DATASET
full_df = pd.concat([agg_200_300, agg_200_1000, agg_200_25000])
test_df = agg_200_5000

# Load the diabetes dataset
features = ['packet_size_total', 'pck_ct', 'packet_sizes_var', 'Time']

# Use only one feature
#diabetes_X = diabetes_X[:, np.newaxis, 2]

# Split the data into training/testing sets
X_train = full_df[features]
y_train = full_df['ratio']
X_test = test_df[features]
y_test = test_df['ratio']

# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = regr.predict(X_test)

# The coefficients
print("Coefficients: \n", regr.coef_)
# The mean squared error
print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred))
# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(y_test, y_pred))

Coefficients: 
 [-3.53794596e-04 -2.07567811e-04  1.70578139e-06  8.87861252e-07]
Mean squared error: 22.28
Coefficient of determination: 0.00


In [94]:
###ENTIRE DATASET
full_df = pd.concat([agg_200_300, agg_200_1000, agg_200_5000, agg_200_25000])

# Load the diabetes dataset
features = ['packet_size_total', 'pck_ct', 'packet_sizes_var', 'Time']
df_X = full_df[features]
df_y = full_df['ratio']

# Use only one feature
#diabetes_X = diabetes_X[:, np.newaxis, 2]

# Split the data into training/testing sets
X_train, X_rem, y_train, y_rem = train_test_split(df_X, df_y, train_size=0.8, random_state=42)

X_valid, X_test, y_valid, y_test = train_test_split(X_rem, y_rem, test_size=0.5)

# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = regr.predict(X_test)

# The coefficients
print("Coefficients: \n", regr.coef_)
# The mean squared error
print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred))
# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(y_test, y_pred))

Coefficients: 
 [-4.89551417e-04  4.51564168e-04  7.53356081e-07  8.14791660e-08]
Mean squared error: 0.05
Coefficient of determination: 0.30


In [95]:
# Plot outputs
# plt.scatter(X_test, y_test, color="black")
# plt.plot(X_test, y_pred, color="blue", linewidth=3)

# plt.xticks(())
# plt.yticks(())

# plt.show()

In [96]:
# def model_w_features(df):
#     dummy = df 
    
#     #
#     dummy['']

In [97]:
agg_200_300.head()

,1->2Bytes,2->1Bytes,1->2Pkts,2->1Pkts,packet_size_total,pck_ct,packet_sizes_var,Time,ratio
0,1444556,12592,964,235,1199,1199,1536.600000,1635274148,0.67
1,1446000,15172,964,283,1247,1247,196.677778,1635274158,0.67
2,1441500,15228,961,285,1246,1246,173.822222,1635274168,0.67
3,1456500,16052,971,299,1270,1270,108.222222,1635274178,0.67
4,1434056,15088,957,280,1237,1237,1587.072727,1635274188,0.67


In [98]:
pd.Series(run_200_300['packet_times'][25].split(';')[:-1]).astype(int).diff().mean()

8.907407407407407

In [99]:
run_200_300

,Time,IP1,Port1,IP2,Port2,Proto,1->2Bytes,2->1Bytes,1->2Pkts,2->1Pkts,packet_times,packet_sizes,packet_dirs,pck_ct,packet_size_total,ratio
25,1635274148,192.168.48.3,59088,172.31.0.2,5001,6,126000,1300,84,25,1635274148018;1635274148018;1635274148018;1635...,52;1500;1500;52;1500;52;1500;1500;1500;1500;52...,2;1;1;2;1;2;1;1;1;1;2;1;1;1;1;2;1;1;1;1;1;1;1;...,109,109,0.67
26,1635274149,192.168.48.3,59088,172.31.0.2,5001,6,178500,1612,119,31,1635274149005;1635274149005;1635274149005;1635...,52;1500;1500;1500;1500;52;1500;1500;1500;1500;...,2;1;1;1;1;2;1;1;1;1;1;1;2;1;1;1;1;2;1;1;2;1;1;...,150,150,0.67
27,1635274150,192.168.48.3,59088,172.31.0.2,5001,6,118500,1212,79,21,1635274150032;1635274150032;1635274150032;1635...,52;1500;1500;52;1500;1500;1500;1500;1500;1500;...,2;1;1;2;1;1;1;1;1;1;2;1;1;1;1;2;1;1;1;1;1;1;1;...,100,100,0.67
28,1635274151,192.168.48.3,59088,172.31.0.2,5001,6,147000,1040,98,20,1635274151020;1635274151020;1635274151020;1635...,52;1500;1500;1500;1500;52;1500;1500;52;1500;15...,2;1;1;1;1;2;1;1;2;1;1;2;1;1;1;1;1;1;1;1;1;1;2;...,118,118,0.67
29,1635274152,192.168.48.3,59088,172.31.0.2,5001,6,138000,1144,92,22,1635274152045;1635274152045;1635274152045;1635...,52;1500;1500;1500;1500;1500;1500;1500;1500;52;...,2;1;1;1;1;1;1;1;1;2;1;1;1;1;1;1;2;1;1;1;1;1;1;...,114,114,0.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319,1635274438,192.168.48.3,59088,172.31.0.2,5001,6,160500,1612,107,31,1635274438095;1635274438095;1635274438095;1635...,52;1500;1500;1500;1500;1500;1500;52;1500;1500;...,2;1;1;1;1;1;1;2;1;1;2;1;1;2;1;1;1;1;2;1;1;1;2;...,138,138,0.67
320,1635274439,192.168.48.3,59088,172.31.0.2,5001,6,153000,1764,102,33,1635274439060;1635274439060;1635274439060;1635...,52;1500;1500;1500;1500;1500;1500;52;1500;1500;...,2;1;1;1;1;1;1;2;1;1;1;1;2;1;1;2;1;1;2;1;1;2;1;...,135,135,0.67
321,1635274440,192.168.48.3,59088,172.31.0.2,5001,6,127500,1356,85,24,1635274440104;1635274440104;1635274440104;1635...,64;1500;1500;1500;1500;64;1500;1500;64;1500;15...,2;1;1;1;1;2;1;1;2;1;1;1;1;1;1;2;1;1;1;1;2;1;2;...,109,109,0.67
322,1635274441,192.168.48.3,59088,172.31.0.2,5001,6,150000,1300,100,25,1635274441050;1635274441050;1635274441050;1635...,52;1500;1500;52;1500;1500;1500;1500;52;1500;15...,2;1;1;2;1;1;1;1;2;1;1;1;1;2;1;1;2;1;1;1;1;2;1;...,125,125,0.67


<h3>Average Time Delta</h3>

In [100]:
def avg_time_delt(a):
    return pd.Series(a.split(';')[:-1]).astype(int).diff().mean()

In [101]:
def agg_10sec(df):
    new_df = pd.DataFrame()
    min_time = df["Time"][25]
    ratio = df["ratio"][25]
    while min_time < df["Time"][len(df)-1]:
        temp_df = df[(df["Time"] >= min_time) & (df["Time"] < min_time+10)]
        row = temp_df[["1->2Bytes", "2->1Bytes", "1->2Pkts", "2->1Pkts", "packet_size_total", "packet_times","pck_ct"]].sum().to_frame().T
        row["packet_sizes_var"] = temp_df["packet_size_total"].var()
        row['avg_time_delta'] = temp_df['packet_times'].apply(avg_time_delt).mean()
        row["Time"] = min_time
        row["ratio"] = ratio
        new_df = new_df.append(row)
        min_time += 10
    return new_df.reset_index(drop=True)

In [102]:
agg_200_300 = agg_10sec(run_200_300)
agg_200_1000 = agg_10sec(run_200_1000)
agg_200_5000 = agg_10sec(run_200_5000)
agg_200_25000 = agg_10sec(run_200_25000)

In [103]:
full_df = pd.concat([agg_200_300, agg_200_1000, agg_200_25000])
test_df = agg_200_5000

In [104]:
full_df['avg_time_delta'].describe()

count    80.000000
mean      5.151911
std       1.886091
min       2.779692
25%       2.968201
50%       5.052313
75%       7.313561
max       7.881562
Name: avg_time_delta, dtype: float64

In [105]:
###ENTIRE DATASET
full_df = pd.concat([agg_200_300, agg_200_1000, agg_200_5000, agg_200_25000])

# Load the diabetes dataset
features = ['packet_size_total', 'pck_ct', 'packet_sizes_var','avg_time_delta', 'Time']
df_X = full_df[features]
df_y = full_df['ratio']

# Use only one feature
#diabetes_X = diabetes_X[:, np.newaxis, 2]

# Split the data into training/testing sets
X_train, X_rem, y_train, y_rem = train_test_split(df_X, df_y, train_size=0.8, random_state=42)

X_valid, X_test, y_valid, y_test = train_test_split(X_rem, y_rem, test_size=0.5)

# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = regr.predict(X_test)

# The coefficients
print("Coefficients: \n", regr.coef_)
# The mean squared error
print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred))
# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(y_test, y_pred))

Coefficients: 
 [-9.89805232e-05  1.93175533e-04  2.38680013e-08  2.36734398e-01
  5.14637307e-07]
Mean squared error: 0.00
Coefficient of determination: 0.98


Average time delta with standard features being tested on seen data

In [106]:
###ENTIRE DATASET
full_df = pd.concat([agg_200_300, agg_200_1000,agg_200_5000])

# Load the diabetes dataset
features = ['packet_size_total', 'pck_ct', 'packet_sizes_var','avg_time_delta', 'Time']
df_X = full_df[features]
df_y = full_df['ratio']

# Use only one feature
#diabetes_X = diabetes_X[:, np.newaxis, 2]

# Split the data into training/testing sets
X_train, X_rem, y_train, y_rem = train_test_split(df_X, df_y, train_size=0.8, random_state=42)

# new data 

full_df = pd.concat([agg_200_25000])

# Load the diabetes dataset
features = ['packet_size_total', 'pck_ct', 'packet_sizes_var','avg_time_delta', 'Time']
df_X = full_df[features]
df_y = full_df['ratio']

filler, X_rem, filler, y_rem = train_test_split(df_X, df_y, train_size=0.8, random_state=42)

X_valid, X_test, y_valid, y_test = train_test_split(X_rem, y_rem, test_size=0.5)

# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = regr.predict(X_test)

# The coefficients
print("Coefficients: \n", regr.coef_)
# The mean squared error
print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred))
# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(y_test, y_pred))

Coefficients: 
 [-5.23771103e-05  1.86082304e-05  3.72704648e-09  4.65740735e-02
 -2.37155784e-07]
Mean squared error: 0.37
Coefficient of determination: 0.00


Average time delta with standard features being tested on unseen data  -- not too many training options doe 

In [111]:
agg_200_300 = agg_10sec(run_200_300)
agg_200_1000 = agg_10sec(run_200_1000)
agg_200_5000 = agg_10sec(run_200_5000)
agg_200_25000 = agg_10sec(run_200_25000)
agg_200_500 = agg_10sec(run_200_500)
agg_200_750 = agg_10sec(run_200_750)
agg_200_200 = agg_10sec(run_200_200)
agg_200_400 = agg_10sec(run_200_400)
agg_200_600 = agg_10sec(run_200_600)
agg_200_800 = agg_10sec(run_200_800)
agg_200_1000 =  agg_10sec(run_200_1000)
agg_200_2000 =  agg_10sec(run_200_2000)
agg_200_1400 =  agg_10sec(run_200_1400)
agg_200_1800 =  agg_10sec(run_200_1800)
agg_200_2500 =  agg_10sec(run_200_2500)

In [115]:
###ENTIRE DATASET
full_df = pd.concat([agg_200_300, agg_200_1000, agg_200_5000, agg_200_25000, agg_200_500, agg_200_750, agg_200_200, agg_200_600, agg_200_1000, agg_200_2000])

# Load the diabetes dataset
features = ['packet_size_total', 'pck_ct', 'packet_sizes_var','avg_time_delta', 'Time']
df_X = full_df[features]
df_y = full_df['ratio']

# Use only one feature
#diabetes_X = diabetes_X[:, np.newaxis, 2]

# Split the data into training/testing sets
X_train, X_rem, y_train, y_rem = train_test_split(df_X, df_y, train_size=0.8, random_state=42)

# new data 

full_df = pd.concat([agg_200_400, agg_200_800, agg_200_1400, agg_200_1800])

# Load the diabetes dataset
features = ['packet_size_total', 'pck_ct', 'packet_sizes_var','avg_time_delta', 'Time']
df_X = full_df[features]
df_y = full_df['ratio']

filler, X_rem, filler, y_rem = train_test_split(df_X, df_y, train_size=0.8, random_state=42)

X_valid, X_test, y_valid, y_test = train_test_split(X_rem, y_rem, test_size=0.5)

# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = regr.predict(X_test)

# The coefficients
print("Coefficients: \n", regr.coef_)
# The mean squared error
print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred))
# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(y_test, y_pred))

Coefficients: 
 [1.35866653e-05 3.48743321e-05 8.52586909e-08 1.60830531e-01
 2.67898689e-07]
Mean squared error: 0.00
Coefficient of determination: 0.97


In [117]:
df_X

,packet_size_total,pck_ct,packet_sizes_var,avg_time_delta,Time
0,2277,1912,1440.200000,5.116142,1636068394
1,2239,1903,706.988889,5.162461,1636068404
2,2114,1775,1296.266667,5.516463,1636068414
3,2310,1769,1896.200000,5.392686,1636068424
4,2203,1795,325.122222,5.429337,1636068434
...,...,...,...,...,...
21,4695,5498,2640.722222,1.779694,1636072651
22,4690,5565,1955.777778,1.791671,1636072661
23,4844,5431,1206.044444,1.821879,1636072671
24,4858,5469,2137.288889,1.799605,1636072681


<h3>Directionality Ratio</h3>